In [1]:
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
# Read in small_data labels and data 
small_images_all_data = pd.read_pickle("/Users/josephjamison/Downloads/small_images_all_data.pkl")
small_images_labels = pd.read_pickle("small_images_labels.pkl")

In [3]:
small_images_labels.columns = ['label']

In [138]:
# Select 5 labels that have 800 examples in large photo bank,then extract only 
# data rows with these labels for POC dataset
np.random.seed(42)

labels = small_images_labels.to_numpy()
POC_labels_choice = ['skyscraper', 'bathroom', 'dining_room', 'airport_terminal', 'highway']
# set(np.random.choice(np.unique(labels).squeeze(), 3))
print(POC_labels_choice)

keep_idx = small_images_labels['label'].isin(POC_labels_choice)
# Drop 200 images from each class to match number of examples of large images 
for i in range(keep_idx.shape[0]):
    if i % 1000 >= 800:
        keep_idx.iloc[i] = False 

POC_small_labels = small_images_labels.loc[keep_idx == True]
POC_small_data = small_images_all_data.loc[keep_idx == True]

print(POC_small_labels.shape)
print(POC_small_data.shape)

['skyscraper', 'bathroom', 'dining_room', 'airport_terminal', 'highway']
(4000, 1)
(4000, 49152)


In [139]:
# Write POC small_data datasets to .pkl
POC_small_labels.to_pickle('5k_POC_small_labels.pkl')
POC_small_data.to_pickle('5k_POC_small_data.pkl')

In [140]:
# Retrieve 1000 large images for each label in POC_labels_choice

path = '/Users/josephjamison/Downloads/SUN397/'

images = []
labels = []

for label in POC_labels_choice:
    counter = 0
    first_letter = label[0]    
    dir_list = [i for i in os.listdir(os.path.join(path, first_letter, label)) if not i.startswith('.')]
    for file in dir_list:
        if counter < 800:
            try:
                # Read in image, re-size to 256 x 256, then transpose to (C, H, W)
                im = cv2.imread(os.path.join(path, first_letter, label, file))
                resized_im = cv2.resize(im, (256, 256), interpolation= cv2.INTER_LINEAR).transpose((2, 0, 1))
                images.append(resized_im)
                labels.append(label)
                counter += 1
            except:
                pass

images = np.array(images)
labels = np.array(labels)
print(np.unique(labels))
print(images.shape)
print(labels.shape)
                

libpng warning: iCCP: known incorrect sRGB profile


['airport_terminal' 'bathroom' 'dining_room' 'highway' 'skyscraper']
(4000, 3, 256, 256)
(4000,)


In [147]:
POC_large_labels = pd.DataFrame(labels)
POC_large_labels.columns = ['label']

N = images.shape[0]
POC_large_data = pd.DataFrame(images.reshape(N, -1))

# print(POC_large_labels['label'].value_counts()['bathroom'])
# print(POC_large_data.shape)

In [148]:
# Write POC large_data datasets to .pkl
POC_large_labels.to_pickle('5k_POC_large_labels.pkl')
POC_large_data.to_pickle('5k_POC_large_data.pkl')